In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_DTI_matrixes.ipynb

ERROR:root:File `'../data_DTI_matrixes.ipynb.py'` not found.


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
01_SLD,0,0
02_MSG,0,0
03_JAC,0,0
04_NSV,0,0
05_EGR,0,0


(225, 2)

In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.333, random_state=2018, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(150, 39)
(75, 39)
(150,)
(75,)


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    ('reduce_dim', PCA()),
    ('classify', SVC(kernel='poly',decision_function_shape='ovo'))
])

In [8]:
# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
PENALTY = ['l1', 'l2']
REGULATION_C = np.linspace(1, 500, 50)
MATRIXES = list(all_combinations(['DTI_FA', 'DTI_L1', 'DTI_MD', 'DTI_RX', ]))
EXTRA_1D_COLUMS = [None] # + list(all_combinations(available_columns))                            
DEGREE = range(2, 10)

param_grid = { 
        'combining__columns': MATRIXES,
        'filter__columns': EXTRA_1D_COLUMS,
        
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C' : REGULATION_C,
        'classify__degree': DEGREE,
}


In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [10]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] reduce_dim__n_components=221, filter__columns=None, combining__columns=('DTI_L1',), classify__degree=4, classify__C=296.32653061224494 
[CV] reduce_dim__n_components=221, filter__columns=None, combining__columns=('DTI_L1',), classify__degree=4, classify__C=296.32653061224494 
[CV] reduce_dim__n_components=221, filter__columns=None, combining__columns=('DTI_L1',), classify__degree=4, classify__C=296.32653061224494 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=8, classify__C=214.8571428571429 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=8, classify__C=214.8571428571429 
[CV] reduce_dim__n_components=471, filter__columns=None, combining__columns=('DTI_FA', 'DTI_RX'), classify__degree=8, classify__C=214.8571428571429 
[CV] reduce_dim__n_components=51, filter__columns=None, combining__

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.6/runpy.py in _run_code(code=<

In [ ]:
%run ../grid_results.ipynb